# Read Initial Data

In [45]:
import pandas as pd
time = []
open = []
high = []
low = []
close = []
volume = []
size = -1

def read_df(file_name):

    df = pd.read_feather(file_name)
    df = df.dropna()    # Dropping NA values, so that talib works correctly
    global size
    size = df.shape[0]
    
    global time
    time = df['time'].to_numpy()

    global open
    open = df['open'].to_numpy()

    global high
    high = df['high'].to_numpy()

    global low
    low = df['low'].to_numpy()

    global close
    close = df['close'].to_numpy()

    global volume
    volume = df['volume'].to_numpy()

    return

read_df("data-BNBUSDT1h.feather")

# Create Indicators

In [46]:
import sys
import numpy as np
np.set_printoptions(threshold=sys.maxsize)
import talib as ta    # https://github.com/mrjbq7/ta-lib

ma_direction_period = 28
ma_direction = ta.EMA(close, timeperiod=ma_direction_period)
atr_direction_period = 14
atr_direction = ta.ATR(high, low, close, timeperiod=atr_direction_period)

# Generate Trades

In [47]:
in_trade = 0    # +1 = long_entry, -1 = shor_entry
entry_price = -1.0
trades_long = []
trades_short = []

for i in range(size):
    # Long Entry
    if (ma_direction[i] - close[i] > 4.0*atr_direction[i]
        and in_trade == 0.0):
        in_trade = +1.0
        entry_price = close[i]
    # Long Exit
    if (ma_direction[i] - close[i] < 0.0
        and in_trade == 1.0):
        in_trade = 0.0
        trades_long.append(close[i]/entry_price)

    # Short Entry
    if (ma_direction[i] - close[i] < -4.0*atr_direction[i]
        and in_trade == 0.0):
        in_trade = -1.0
        entry_price = close[i]
    # Short Exit
    if (ma_direction[i] - close[i] > 0.0
        and in_trade == -1.0):
        in_trade = 0.0
        trades_short.append(entry_price/close[i])


# Calculate Results

In [48]:
long_res = 1.0
for k in trades_long:
    long_res *= k

print("Longs result: ", long_res)
average_long_trades_res = np.sum(trades_long)/len(trades_long)
print("Average Long Trade: ", average_long_trades_res)
DD_long = 1.0 - np.min(trades_long)
print("Highest DD for Longs: ", DD_long)
print()

short_res = 1.0
for k in trades_short:
    short_res *= k

print("Shorts result: ", short_res)
average_short_trades_res = np.sum(trades_short)/len(trades_short)
print("Average Short Trade: ", average_short_trades_res)
DD_short = 1.0 - np.min(trades_short)
print("Highest DD for Shorts: ", DD_short)
print()


DD_highest = None
if DD_long > DD_short:
    DD_highest = DD_long
else: DD_highest = DD_short

score = ((average_long_trades_res + average_short_trades_res) / 2) / DD_highest
print("Score: ", score)

Longs result:  1.1932475343208822
Average Long Trade:  1.0058177152451184
Highest DD for Longs:  0.2489020352179997

Shorts result:  0.16694511027625009
Average Short Trade:  0.9839752741179365
Highest DD for Shorts:  0.4806757383373651

Score:  2.0697872085718907


# Optimize Results

In [49]:
# Strategy Function
def strategy_ma(ma_direction_period, atr_direction_period, atr_direction_coeff, show_money = False):
    # Generate Indicators
    ma_direction = ta.EMA(close, timeperiod=ma_direction_period)
    atr_direction = ta.ATR(high, low, close, timeperiod=atr_direction_period)

    # Calculate Trades
    in_trade = 0    # +1 = long_entry, -1 = shor_entry
    entry_price = -1.0
    trades_long = []
    trades_short = []
    # Parse dataframe
    for i in range(size):
        # Long Entry
        if (ma_direction[i] - close[i] > atr_direction_coeff*atr_direction[i]
            and in_trade == 0.0):
            in_trade = +1.0
            entry_price = close[i]
        # Long Exit
        if (ma_direction[i] - close[i] < 0.0
            and in_trade == 1.0):
            in_trade = 0.0
            trades_long.append(close[i]/entry_price)

        # Short Entry
        if (ma_direction[i] - close[i] < -atr_direction_coeff*atr_direction[i]
            and in_trade == 0.0):
            in_trade = -1.0
            entry_price = close[i]
        # Short Exit
        if (ma_direction[i] - close[i] > 0.0
            and in_trade == -1.0):
            in_trade = 0.0
            trades_short.append(entry_price/close[i])

    # Calculating Stats
    # Finding Average Trade Size
    if trades_long and trades_short: # If any trade happened
        average_long_trades_res = np.sum(trades_long)/len(trades_long)
        average_short_trades_res = np.sum(trades_short)/len(trades_short)
        # Finding DD
        DD_long = 1.0 - np.min(trades_long)
        DD_short = 1.0 - np.min(trades_short)
        if show_money:
            print("Average Long Trade: ", average_long_trades_res)
            print("Average Short Trade: ", average_short_trades_res)
            print("Long DD ", DD_long)
            print("Short DD ", DD_short)
        DD_highest = None
        if DD_long > DD_short:
            DD_highest = DD_long
        else: DD_highest = DD_short
        score = ((average_long_trades_res + average_short_trades_res) / 2) / DD_highest
        return score
    else: return None


In [50]:
ma_direction_period = np.arange(8, 256, 4)
atr_direction_period = np.arange(4, 64, 2)
atr_direction_coeff = np.linspace(-2.0, 6.0, 100)

ma_direction_period_value = np.random.choice(ma_direction_period)
print(ma_direction_period_value)
atr_direction_period_value = np.random.choice(atr_direction_period)
print(atr_direction_period_value)
atr_direction_coeff_value = np.random.choice(atr_direction_coeff)
print(atr_direction_coeff_value)

152
34
-2.0


# Simulated Annealing

In [51]:
# Cooling Schedules
temp_max = 1.5
temp_min = 0.0
outer_loop = 100
inner_loop = 20
def cool_lin_a(step):
    return temp_min + (temp_max - temp_min) * ((outer_loop - step)/outer_loop)
def cool_quad_a(step):
    return temp_min + (temp_max - temp_min) * ((outer_loop - step)/outer_loop)**2
def cool_exp_a(step):
    return temp_min + (temp_max - temp_min) * ((outer_loop - step)/outer_loop)**np.sqrt(step)
def cool_cos_a(step):
    return temp_min + 0.5*(temp_max - temp_min) * (1 + np.cos(step*np.pi/outer_loop))

# Simulated Annealing
def annealing_solution():

    best_solution = 9.9     # Just a big number to minimize it
    step = 0
    cooling_arr = []
    temp = temp_max

    # Generate Init Candidate Solution
    ma_direction_period_value = np.random.choice(ma_direction_period)
    atr_direction_period_value = np.random.choice(atr_direction_period)
    atr_direction_coeff_value = np.random.choice(atr_direction_coeff)

    current_energy = strategy_ma(ma_direction_period_value, atr_direction_period_value, atr_direction_coeff_value)
    best_params = [ma_direction_period_value, atr_direction_period_value, atr_direction_coeff_value]

    # Annealing Schedule
    while step < outer_loop:
        
        step += 1
        mc_step = 0

        while mc_step < inner_loop:

            mc_step += 1

            # Generate Candidate Solution
            ma_direction_period_value = np.random.choice(ma_direction_period)
            atr_direction_period_value = np.random.choice(atr_direction_period)
            atr_direction_coeff_value = np.random.choice(atr_direction_coeff)

            candidate_energy = strategy_ma(ma_direction_period_value, atr_direction_period_value, atr_direction_coeff_value)
            # In a correct solution candidate energy should be saved to be iterated upon
            if candidate_energy:    # If any solution found
                delta_energy = candidate_energy - current_energy    # Positive if solution is better, and is always accepted
                if candidate_energy < best_solution:
                    best_solution = candidate_energy
                    best_params = [ma_direction_period_value, atr_direction_period_value, atr_direction_coeff_value]

                if np.random.random() < np.exp(-delta_energy / temp):   # Value is >1 if solution is better anyway, otherwise Metropolis is used
                    current_energy = candidate_energy
            
        # Cooling Function
        temp = cool_quad_a(step)
        
    return best_params

found_sol = annealing_solution()
print(strategy_ma(found_sol[0], found_sol[1], found_sol[2], show_money=True))

Average Long Trade:  1.1400349446709377
Average Short Trade:  1.025509912160195
Long DD  -0.1400349446709377
Short DD  -0.020691333982473248
-52.32975454036637
